In [ ]:
# This nodebook extracts poses from a data source

%load_ext autoreload
%autoreload 2

In [ ]:
from pretrained import BodyDetector, Pose

In [ ]:
from PIL import Image
from IPython.display import display
from pathlib import Path
from tqdm import tqdm

from constants import emotions

processed_dir = Path("/media/regieki/data/RegieKI/data/processed4")
device_paths = [f for f in processed_dir.iterdir() if f.is_dir()]


In [ ]:

data_dir = Path("/media/regieki/data/RegieKI/data/all_faces2") 


detector = BodyDetector()

for emotion in emotions:
    i = 0
    for device_path in device_paths:
        device = device_path.stem
        print(f"Starting {device} -> {emotion}")
        image_dir  = Path(device_path, emotion)

        for image_path in tqdm(image_dir.glob("*.png")):
            output_dir = Path(data_dir, device, emotion)

            print(output_dir.resolve())
            
            if(not output_dir.exists()):
                output_dir.mkdir(parents=True, exist_ok=True)

            face_path  = Path(output_dir, f"image_{i:06d}.jpg")

            if( face_path.exists()):
                continue

            img = Image.open(image_path).convert("RGB")
            pose = detector.get_pose(img)

            for face in faces:                
                i += 1
                face.save(face_path)

In [ ]:
# from pycoral.adapters import common
# from pycoral.adapters import detect
# from pycoral.utils.dataset import read_label_file
# from pycoral.utils.edgetpu import make_interpreter

#import tflite_runtime.interpreter as tflite
import pprint
import tensorflow.lite as tflite
import numpy as np

EDGETPU_SHARED_LIB = 'libedgetpu.so.1'
POSENET_SHARED_LIB = "/home/regieki/regieki-core/pretrained/posenet/posenet_decoder.so"

edgetpu_delegate         = tf.lite.experimental.load_delegate(EDGETPU_SHARED_LIB)
posenet_decoder_delegate = tf.lite.experimental.load_delegate(POSENET_SHARED_LIB)

delegates = [
    edgetpu_delegate,     
    posenet_decoder_delegate
]


pp = pprint.PrettyPrinter(indent=4)

In [ ]:
posenet     = "../pretrained/posenet/posenet_mobilenet_v1_075_481_641_quant_decoder_edgetpu.tflite"

In [ ]:
import enum
import collections

class KeypointType(enum.IntEnum):
  """Pose kepoints."""
  NOSE = 0
  LEFT_EYE = 1
  RIGHT_EYE = 2
  LEFT_EAR = 3
  RIGHT_EAR = 4
  LEFT_SHOULDER = 5
  RIGHT_SHOULDER = 6
  LEFT_ELBOW = 7
  RIGHT_ELBOW = 8
  LEFT_WRIST = 9
  RIGHT_WRIST = 10
  LEFT_HIP = 11
  RIGHT_HIP = 12
  LEFT_KNEE = 13
  RIGHT_KNEE = 14
  LEFT_ANKLE = 15
  RIGHT_ANKLE = 16


In [ ]:
# interpreter = make_interpreter(resnet)
# interpreter.allocate_tensors()

from PIL import Image

interpreter =  tflite.Interpreter(
    model_path=posenet,
    experimental_delegates=delegates)

interpreter.allocate_tensors()

num_keypoints = int(KeypointType.RIGHT_ANKLE + 1)

Pose = collections.namedtuple('Pose', ['keypoints', 'score'])

size = (641, 481)

image_path = "../tmp/test.png"
image = Image.open(image_path).convert("RGB").resize(size)
input_img = np.expand_dims(np.asarray(image), axis=0)

for model in test_models:
    print(model)




    #print("inference %s" % output())

    print(pose)

In [ ]:
for pose in poses:
    print(pose)

In [ ]:
# face_landmarks_pb = "/media/meredityman/DATA/Projects/RegieKi/Code/Production/regieki-core/pretrained/face_landmark_shufflenetv2_1.0-20201204T150037Z-001/face_landmark_shufflenetv2_1.0/keypoints/"

# face_landmarks_tflite = "/media/meredityman/DATA/Projects/RegieKi/Code/Production/regieki-core/pretrained/face_landmark_shufflenetv2_1.0-20201204T150037Z-001/face_landmark_shufflenetv2_1.0/keypoints.tflite"


# converter = tf.lite.TFLiteConverter.from_saved_model(face_landmarks_pb)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# tflite_quant_model = converter.convert()


# # Save the model.
# with open(face_landmarks_tflite, 'wb') as f:
#   f.write(tflite_quant_model)


In [ ]:



interpreter =  tflite.Interpreter(
    model_path=face_landmarks,
    experimental_delegates=delegates)

interpreter.allocate_tensors()


# for info in interpreter.get_tensor_details():
#     print(f"{info['name']}")

input = interpreter.tensor(interpreter.get_input_details()[0]["index"])
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])

print(interpreter.get_input_details()[0]["index"])
print(interpreter.get_output_details()[0]["index"])

for i in range(10):
    input().fill(0.5)
    interpreter.invoke()
    print("inference %s" % output())
